In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
import requests

In [2]:
# elevation data https://en-au.topographic-map.com/place-q9knh/Florida/
# Data is in meters
data = {
    'NAME': ['Osceola', 'Orange', 'Okeechobee', 'Palm Beach', 'Putnam', 'Lake', 
               'Lafayette', 'IndianRiver', 'Hardee', 'Glades', 'Gilchrist', 'Gulf', 'Pinellas', 'Highlands',
               'Hillsborough', 'Hendry', 'Hernando', 'Union', 'Taylor', 'Collier', 'DeSoto', 'Dixie', 'Sarasota',
               'Seminole', 'St. Lucie', 'Sumter', 'Levy', 'Lee', 'Wakulla', 'St. Johns', 'Volusia', 'Pasco', 'Polk',
               'Franklin', 'Flagler', 'Brevard', 'Bradford', 'Alachua', 'Clay', 'Broward', 'Charlotte', 'Citrus',
               'Monroe', 'Miami-Dade', 'Martin', 'Marion', 'Manatee', 'Jackson', 'Holmes', 'Okaloosa', 'Jefferson', 
               'Hamilton', 'Washington', 'Suwannee', 'Columbia', 'Santa Rosa', 'Madison', 'Liberty', 'Leon', 
               'Escambia', 'Duval', 'Gadsden', 'Bay', 'Baker', 'Walton', 'Calhoun', 'Nassau' ],
    'AVG ELEVATION': [19, 112, 12, 5, 17, 56, 22, 8, 8, 12, 19, 5, 4, 22, 20, 8, 18, 41, 8, 5, 18, 7, 7, 14, 8,
                     27, 11, 5, 10, 6, 8, 18, 33, 4, 6, 9, 45, 33, 29, 5, 10, 14, 1, 3, 6, 24, 13, 44, 39, 37, 23,
                     37, 31, 27, 36, 32, 33, 27, 38, 30, 11, 50, 13, 40, 31, 31, 13],
    'INCOME_GROWTH_PERCENTAGES' : [0.015, 0.0186, 0.0166, 0.0273, 0.0193, 0.0167, 0.019, 0.03, 0.0145, 0.0184, 0.0199, 
                                  0.0243, 0.0211, 0.0153, 0.026, 0.0079, 0.0198, 0.0186, 0.0163, 0.0288, 0.0150, 0.0181,
                                  0.0213, 0.024, 0.0192, 0.0311, 0.02, 0.0216, 0.0233, 0.0259, 0.0196, 0.0212, 0.0166, 
                                  0.0257, 0.0225, 0.0152, 0.024, 0.0219, 0.0156, 0.0181, 0.0199, 0.0202, 0.0323, 0.0199,
                                  0.0292, 0.0196, 0.021, 0.0222, 0.0228, 0.0202, 0.026, 0.0197, 0.0217, 0.0191, 0.0192, 
                                  0.0191, 0.0217, 0.0174, 0.0215, 0.0178, 0.0183, 0.0264, 0.0218, 0.0192, 0.0351, 0.0225,
                                  0.0264]
}

elevation_df = pd.DataFrame(data)

print(elevation_df)

          NAME  AVG ELEVATION  INCOME_GROWTH_PERCENTAGES
0      Osceola             19                     0.0150
1       Orange            112                     0.0186
2   Okeechobee             12                     0.0166
3   Palm Beach              5                     0.0273
4       Putnam             17                     0.0193
..         ...            ...                        ...
62         Bay             13                     0.0218
63       Baker             40                     0.0192
64      Walton             31                     0.0351
65     Calhoun             31                     0.0225
66      Nassau             13                     0.0264

[67 rows x 3 columns]


In [3]:
url='https://api.census.gov/data/2020/dec/dhc'
params = {
    "get": 'NAME,P1_001N', 
    "for": "county:*",
    "in": "state:12",
    "key": "API_KEY"}
response = requests.get(url,params=params)
total_population=pd.DataFrame(response.json()[1:], columns=response.json()[0])
total_population['NAME']= total_population['NAME'].str.split(' ').str[0]
total_population['P1_001N'].unique().sort()

In [4]:
total_population[50:60]

,NAME,P1_001N,state,county
50,Pasco,561891,12,101
51,Pinellas,959107,12,103
52,Polk,725046,12,105
53,Putnam,73321,12,107
54,Sarasota,434006,12,115
55,St.,273425,12,109
56,St.,329226,12,111
57,Santa,188000,12,113
58,Seminole,470856,12,117
59,Sumter,129752,12,119


In [5]:
total_population['NAME'].unique()

array(['Alachua', 'Baker', 'Bay', 'Bradford', 'Brevard', 'Broward',
       'Calhoun', 'Charlotte', 'Citrus', 'Clay', 'Collier', 'Columbia',
       'DeSoto', 'Dixie', 'Duval', 'Escambia', 'Flagler', 'Franklin',
       'Gadsden', 'Gilchrist', 'Glades', 'Gulf', 'Hamilton', 'Hardee',
       'Hendry', 'Hernando', 'Highlands', 'Hillsborough', 'Holmes',
       'Indian', 'Jackson', 'Jefferson', 'Lafayette', 'Lake', 'Lee',
       'Leon', 'Levy', 'Liberty', 'Madison', 'Manatee', 'Marion',
       'Martin', 'Miami-Dade', 'Monroe', 'Nassau', 'Okaloosa',
       'Okeechobee', 'Orange', 'Osceola', 'Palm', 'Pasco', 'Pinellas',
       'Polk', 'Putnam', 'Sarasota', 'St.', 'Santa', 'Seminole', 'Sumter',
       'Suwannee', 'Taylor', 'Union', 'Volusia', 'Wakulla', 'Walton',
       'Washington'], dtype=object)

In [6]:
# Income data
income_data = pd.read_excel('/Users/ehast/Downloads/AllCounties_Data.xlsx')
# Rename columns for clarity
income_data.rename(columns={
    'Median Household Income, Dollars, 2010': 'NAME',
    'Unnamed: 1': 'Income/year'
}, inplace=True)

In [7]:
# Drop the first two row
income_data = income_data.drop(0)
income_data = income_data.drop(1)
# Reset the index
income_data.reset_index(drop=True, inplace=True)

In [8]:
# Corrections to be made
corrections = {
    "Indian": "IndianRiver",
    "Palm": "Palm Beach",
    "St.": "St. Johns",
    "Santa": "Santa Rosa"
}

# Insert 'St. Lucie' after 'St. Johns'
st_johns_index = total_population[total_population['NAME'] == 'St.'].index[0]
st_lucie_row = pd.DataFrame([{'NAME': 'St. Lucie', 'P1_001N': 0, 'state': 12, 'county': '000'}])
total_population = pd.concat([total_population.iloc[:st_johns_index+1], st_lucie_row, total_population.iloc[st_johns_index+1:]]).reset_index(drop=True)

# Apply corrections
total_population['NAME'] = total_population['NAME'].replace(corrections)

In [9]:
total_population

,NAME,P1_001N,state,county
0,Alachua,278468,12,001
1,Baker,28259,12,003
2,Bay,175216,12,005
3,Bradford,28303,12,007
4,Brevard,606612,12,009
...,...,...,...,...
63,Union,16147,12,125
64,Volusia,553543,12,127
65,Wakulla,33764,12,129
66,Walton,75305,12,131


In [10]:
# Remove the incorrect 'St. Johns' row
# The second 'St. Johns' entry is at index 57
total_population = total_population.drop(index=57)

# Correct the population number for 'St. Lucie'
# 'St. Lucie' is currently at index 56
total_population.loc[total_population['NAME'] == 'St. Lucie', 'P1_001N'] = 329226

# Reset index after the removal
total_population.reset_index(drop=True, inplace=True)

total_population[50:60]

,NAME,P1_001N,state,county
50,Pasco,561891,12,101
51,Pinellas,959107,12,103
52,Polk,725046,12,105
53,Putnam,73321,12,107
54,Sarasota,434006,12,115
55,St. Johns,273425,12,109
56,St. Lucie,329226,12,000
57,Santa Rosa,188000,12,113
58,Seminole,470856,12,117
59,Sumter,129752,12,119


In [11]:
merged_data = total_population.merge(income_data, on='NAME', how='inner')
merged_data

,NAME,P1_001N,state,county,Income/year
0,Alachua,278468,12,001,40644
1,Baker,28259,12,003,47276
2,Bay,175216,12,005,47770
3,Bradford,28303,12,007,41126
4,Brevard,606612,12,009,49523
...,...,...,...,...,...
62,Union,16147,12,125,41794
63,Volusia,553543,12,127,44400
64,Wakulla,33764,12,129,53301
65,Walton,75305,12,131,47273


In [12]:
merged_data['Income/year'] = merged_data['Income/year'].astype(float)
merged_data['P1_001N'] = merged_data['P1_001N'].astype(float)

In [13]:
merged_data

,NAME,P1_001N,state,county,Income/year
0,Alachua,278468.0,12,001,40644.0
1,Baker,28259.0,12,003,47276.0
2,Bay,175216.0,12,005,47770.0
3,Bradford,28303.0,12,007,41126.0
4,Brevard,606612.0,12,009,49523.0
...,...,...,...,...,...
62,Union,16147.0,12,125,41794.0
63,Volusia,553543.0,12,127,44400.0
64,Wakulla,33764.0,12,129,53301.0
65,Walton,75305.0,12,131,47273.0


In [14]:
merged_data = merged_data.merge(elevation_df, on='NAME', how='inner')
merged_data = merged_data.drop(['state','county'], axis=1)

In [15]:
merged_data

,NAME,P1_001N,Income/year,AVG ELEVATION,INCOME_GROWTH_PERCENTAGES
0,Alachua,278468.0,40644.0,33,0.0219
1,Baker,28259.0,47276.0,40,0.0192
2,Bay,175216.0,47770.0,13,0.0218
3,Bradford,28303.0,41126.0,45,0.0240
4,Brevard,606612.0,49523.0,9,0.0152
...,...,...,...,...,...
62,Union,16147.0,41794.0,41,0.0186
63,Volusia,553543.0,44400.0,8,0.0196
64,Wakulla,33764.0,53301.0,10,0.0233
65,Walton,75305.0,47273.0,31,0.0351


In [16]:
merged_data['elevation_post_rise'] = merged_data['AVG ELEVATION']-1.92032 
# This estimate comes from https://climatecenter.fsu.edu/topics/sea-level-rise#:~:text=along%20the%20coasts.-,Historical%20Trends%20in%20Sea%20Levels,rate%20of%20sea%20level%20rise.
# only run once per attempt, otherwise it will cause issues.

In [17]:
merged_data[merged_data['NAME'] == 'Miami-Dade'] # Value to check, if correct, elevation post rise will be 1.07968.

,NAME,P1_001N,Income/year,AVG ELEVATION,INCOME_GROWTH_PERCENTAGES,elevation_post_rise
42,Miami-Dade,2701767.0,43605.0,3,0.0199,1.07968


In [18]:
bins = [0, 50000, 100000, 500000, 1000000, 2500000, 100000000000]
labels = [0, 1, 2, 3, 4, 5]
merged_data['Population_level'] = pd.cut(merged_data["P1_001N"], bins = bins, labels = labels, include_lowest= True)
merged_data

,NAME,P1_001N,Income/year,AVG ELEVATION,INCOME_GROWTH_PERCENTAGES,elevation_post_rise,Population_level
0,Alachua,278468.0,40644.0,33,0.0219,31.07968,2
1,Baker,28259.0,47276.0,40,0.0192,38.07968,0
2,Bay,175216.0,47770.0,13,0.0218,11.07968,2
3,Bradford,28303.0,41126.0,45,0.0240,43.07968,0
4,Brevard,606612.0,49523.0,9,0.0152,7.07968,3
...,...,...,...,...,...,...,...
62,Union,16147.0,41794.0,41,0.0186,39.07968,0
63,Volusia,553543.0,44400.0,8,0.0196,6.07968,3
64,Wakulla,33764.0,53301.0,10,0.0233,8.07968,0
65,Walton,75305.0,47273.0,31,0.0351,29.07968,1


In [19]:
merged_data['Income/year'].unique()

array([40644., 47276., 47770., 41126., 49523., 51694., 31699., 45037.,
       37933., 61185., 58106., 38214., 35979., 32312., 49463., 43573.,
       48090., 36490., 35728., 37039., 39429., 39178., 37613., 37466.,
       37298., 42011., 34946., 49536., 32247., 47341., 38257., 41359.,
       46445., 46477., 50014., 44490., 35737., 40777., 37459., 47812.,
       40339., 53210., 43605., 53821., 58712., 54242., 38339., 50138.,
       46328., 53242., 44228., 45258., 43946., 34645., 49388., 62663.,
       45196., 55129., 58971., 43079., 36352., 37408., 41794., 44400.,
       53301., 47273., 36216.])

In [20]:
bins = [0, 30000, 35000, 40000, 45000, 50000, 100000000000]
labels = [0, 1, 2, 3, 4, 5]
merged_data['Income_level'] = pd.cut(merged_data["Income/year"], bins = bins, labels = labels, include_lowest= True)
merged_data

,NAME,P1_001N,Income/year,AVG ELEVATION,INCOME_GROWTH_PERCENTAGES,elevation_post_rise,Population_level,Income_level
0,Alachua,278468.0,40644.0,33,0.0219,31.07968,2,3
1,Baker,28259.0,47276.0,40,0.0192,38.07968,0,4
2,Bay,175216.0,47770.0,13,0.0218,11.07968,2,4
3,Bradford,28303.0,41126.0,45,0.0240,43.07968,0,3
4,Brevard,606612.0,49523.0,9,0.0152,7.07968,3,4
...,...,...,...,...,...,...,...,...
62,Union,16147.0,41794.0,41,0.0186,39.07968,0,3
63,Volusia,553543.0,44400.0,8,0.0196,6.07968,3,3
64,Wakulla,33764.0,53301.0,10,0.0233,8.07968,0,5
65,Walton,75305.0,47273.0,31,0.0351,29.07968,1,4


In [21]:
bins = [-5, 0, 20, 40, 60, 80, 1000]
labels = [0, 1, 2, 3, 4, 5]
merged_data['Elevation_level'] = pd.cut(merged_data["elevation_post_rise"], bins = bins, labels = labels, include_lowest= True)
merged_data

,NAME,P1_001N,Income/year,AVG ELEVATION,INCOME_GROWTH_PERCENTAGES,elevation_post_rise,Population_level,Income_level,Elevation_level
0,Alachua,278468.0,40644.0,33,0.0219,31.07968,2,3,2
1,Baker,28259.0,47276.0,40,0.0192,38.07968,0,4,2
2,Bay,175216.0,47770.0,13,0.0218,11.07968,2,4,1
3,Bradford,28303.0,41126.0,45,0.0240,43.07968,0,3,3
4,Brevard,606612.0,49523.0,9,0.0152,7.07968,3,4,1
...,...,...,...,...,...,...,...,...,...
62,Union,16147.0,41794.0,41,0.0186,39.07968,0,3,2
63,Volusia,553543.0,44400.0,8,0.0196,6.07968,3,3,1
64,Wakulla,33764.0,53301.0,10,0.0233,8.07968,0,5,1
65,Walton,75305.0,47273.0,31,0.0351,29.07968,1,4,2


In [22]:
merged_data['Income_level'] = merged_data['Income_level'].astype('float32')
merged_data['Population_level'] = merged_data['Population_level'].astype('float32')
merged_data['Elevation_level'] = merged_data['Elevation_level'].astype('float32')

In [23]:
merged_data

,NAME,P1_001N,Income/year,AVG ELEVATION,INCOME_GROWTH_PERCENTAGES,elevation_post_rise,Population_level,Income_level,Elevation_level
0,Alachua,278468.0,40644.0,33,0.0219,31.07968,2.0,3.0,2.0
1,Baker,28259.0,47276.0,40,0.0192,38.07968,0.0,4.0,2.0
2,Bay,175216.0,47770.0,13,0.0218,11.07968,2.0,4.0,1.0
3,Bradford,28303.0,41126.0,45,0.0240,43.07968,0.0,3.0,3.0
4,Brevard,606612.0,49523.0,9,0.0152,7.07968,3.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...
62,Union,16147.0,41794.0,41,0.0186,39.07968,0.0,3.0,2.0
63,Volusia,553543.0,44400.0,8,0.0196,6.07968,3.0,3.0,1.0
64,Wakulla,33764.0,53301.0,10,0.0233,8.07968,0.0,5.0,1.0
65,Walton,75305.0,47273.0,31,0.0351,29.07968,1.0,4.0,2.0


In [24]:
# Define weights
weights = {
    'P1_001N': 10,  # Weight for population
    'elevation_post_rise': 5,  # Weight for elevation
    'Income/year': 5  # Weight for income
}

# Calculate risk score
merged_data['risk_score'] = (
    merged_data['Population_level'] * weights['P1_001N'] + 50 -
    merged_data['Elevation_level'] * weights['elevation_post_rise'] -
    merged_data['Income_level'] * weights['Income/year']
)

In [25]:
merged_data

,NAME,P1_001N,Income/year,AVG ELEVATION,INCOME_GROWTH_PERCENTAGES,elevation_post_rise,Population_level,Income_level,Elevation_level,risk_score
0,Alachua,278468.0,40644.0,33,0.0219,31.07968,2.0,3.0,2.0,45.0
1,Baker,28259.0,47276.0,40,0.0192,38.07968,0.0,4.0,2.0,20.0
2,Bay,175216.0,47770.0,13,0.0218,11.07968,2.0,4.0,1.0,45.0
3,Bradford,28303.0,41126.0,45,0.0240,43.07968,0.0,3.0,3.0,20.0
4,Brevard,606612.0,49523.0,9,0.0152,7.07968,3.0,4.0,1.0,55.0
...,...,...,...,...,...,...,...,...,...,...
62,Union,16147.0,41794.0,41,0.0186,39.07968,0.0,3.0,2.0,25.0
63,Volusia,553543.0,44400.0,8,0.0196,6.07968,3.0,3.0,1.0,60.0
64,Wakulla,33764.0,53301.0,10,0.0233,8.07968,0.0,5.0,1.0,20.0
65,Walton,75305.0,47273.0,31,0.0351,29.07968,1.0,4.0,2.0,30.0


In [26]:
# Set the annual growth rates
annual_population_growth_rate = 0.0128

# Calculate the population and income for each decade until 2150
for year in range(2030, 2160, 10):  # 2030, 2040, ..., 2150
    years_ahead = year - 2020
    
    # Population projections
    merged_data[f'Population_{year}'] = merged_data['P1_001N'] * ((1 + annual_population_growth_rate) ** years_ahead)

    # Income projections
    merged_data[f'Income_{year}'] = merged_data['Income/year'] * ((1 + merged_data['INCOME_GROWTH_PERCENTAGES']) ** years_ahead)

# Display the updated DataFrame
merged_data

,NAME,P1_001N,Income/year,AVG ELEVATION,INCOME_GROWTH_PERCENTAGES,elevation_post_rise,Population_level,Income_level,Elevation_level,risk_score,...,Population_2110,Income_2110,Population_2120,Income_2120,Population_2130,Income_2130,Population_2140,Income_2140,Population_2150,Income_2150
0,Alachua,278468.0,40644.0,33,0.0219,31.07968,2.0,3.0,2.0,45.0,...,8.747977e+05,2.855962e+05,9.934467e+05,3.546797e+05,1.128188e+06,4.404741e+05,1.281204e+06,5.470214e+05,1.454974e+06,6.793418e+05
1,Baker,28259.0,47276.0,40,0.0192,38.07968,0.0,4.0,2.0,20.0,...,8.877468e+04,2.618111e+05,1.008152e+05,3.166520e+05,1.144888e+05,3.829802e+05,1.300169e+05,4.632021e+05,1.476512e+05,5.602278e+05
2,Bay,175216.0,47770.0,13,0.0218,11.07968,2.0,4.0,1.0,45.0,...,5.504351e+05,3.327255e+05,6.250907e+05,4.128051e+05,7.098719e+05,5.121581e+05,8.061519e+05,6.354232e+05,9.154904e+05,7.883554e+05
3,Bradford,28303.0,41126.0,45,0.0240,43.07968,0.0,3.0,3.0,20.0,...,8.891291e+04,3.476263e+05,1.009722e+05,4.406686e+05,1.146671e+05,5.586139e+05,1.302194e+05,7.081272e+05,1.478810e+05,8.976579e+05
4,Brevard,606612.0,49523.0,9,0.0152,7.07968,3.0,4.0,1.0,55.0,...,1.905651e+06,1.925093e+05,2.164115e+06,2.238555e+05,2.457634e+06,2.603059e+05,2.790963e+06,3.026914e+05,3.169502e+06,3.519785e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Union,16147.0,41794.0,41,0.0186,39.07968,0.0,3.0,2.0,25.0,...,5.072525e+04,2.195050e+05,5.760512e+04,2.639254e+05,6.541812e+04,3.173349e+05,7.429079e+04,3.815527e+05,8.436686e+04,4.587661e+05
63,Volusia,553543.0,44400.0,8,0.0196,6.07968,3.0,3.0,1.0,60.0,...,1.738936e+06,2.547225e+05,1.974789e+06,3.092898e+05,2.242630e+06,3.755467e+05,2.546798e+06,4.559972e+05,2.892221e+06,5.536821e+05
64,Wakulla,33764.0,53301.0,10,0.0233,8.07968,0.0,5.0,1.0,20.0,...,1.060685e+05,4.236459e+05,1.204545e+05,5.333751e+05,1.367918e+05,6.715255e+05,1.553449e+05,8.454584e+05,1.764144e+05,1.064442e+06
65,Walton,75305.0,47273.0,31,0.0351,29.07968,1.0,4.0,2.0,30.0,...,2.365681e+05,1.054438e+06,2.686539e+05,1.488826e+06,3.050914e+05,2.102166e+06,3.464710e+05,2.968180e+06,3.934630e+05,4.190958e+06


In [27]:
# Income growth percentage, 1979-2021: https://florida.reaproject.org/analysis/comparative-indicators/growth_by_decade/per_capita_personal_income/reports/#page_3
# Population Growth: http://edr.state.fl.us/content/conferences/population/demographicsummary.pdf

In [28]:
merged_data[merged_data['NAME'] == 'Miami-Dade']

,NAME,P1_001N,Income/year,AVG ELEVATION,INCOME_GROWTH_PERCENTAGES,elevation_post_rise,Population_level,Income_level,Elevation_level,risk_score,...,Population_2110,Income_2110,Population_2120,Income_2120,Population_2130,Income_2130,Population_2140,Income_2140,Population_2150,Income_2150
42,Miami-Dade,2701767.0,43605.0,3,0.0199,1.07968,5.0,3.0,1.0,80.0,...,8.487509e+06,256873.646817,9.638671e+06,312820.689727,1.094597e+07,380952.990445,1.243057e+07,463924.496349,1.411653e+07,564967.184169


In [29]:
# Start with base columns
final_df = merged_data[['NAME', 'AVG ELEVATION']].copy()

# Add decade-specific columns for income and population from merged_data
for year in range(2030, 2160, 10):
    final_df[f'Income_{year}'] = merged_data[f'Income_{year}']
    final_df[f'Population_{year}'] = merged_data[f'Population_{year}']

# Calculate and add estimated elevation for each decade
yearly_decrement = 0.01016
for year in range(2030, 2160, 10):
    total_decrement = ((year - 2020) // 10) * yearly_decrement * 10  # Decade decrement
    final_df[f'elevation_{year}'] = merged_data['AVG ELEVATION'] - total_decrement


In [30]:
final_df

,NAME,AVG ELEVATION,Income_2030,Population_2030,Income_2040,Population_2040,Income_2050,Population_2050,Income_2060,Population_2060,...,elevation_2060,elevation_2070,elevation_2080,elevation_2090,elevation_2100,elevation_2110,elevation_2120,elevation_2130,elevation_2140,elevation_2150
0,Alachua,33,50475.477283,316236.665772,62685.114825,359127.902591,77848.171669,407836.485705,96679.057687,4.631514e+05,...,32.5936,32.492,32.3904,32.2888,32.1872,32.0856,31.984,31.8824,31.7808,31.6792
1,Baker,40,57178.778569,32091.773339,69155.865950,36444.386426,83641.762118,41387.345223,101161.980609,4.700072e+04,...,39.5936,39.492,39.3904,39.2888,39.1872,39.0856,38.984,38.8824,38.7808,38.6792
2,Bay,13,59267.174711,198980.578127,73531.463224,225968.350333,91228.848181,256616.479018,113185.599410,2.914214e+05,...,12.5936,12.492,12.3904,12.2888,12.1872,12.0856,11.984,11.8824,11.7808,11.6792
3,Bradford,45,52133.398585,32141.741067,66086.934008,36501.131286,83775.141563,41451.786399,106197.608486,4.707390e+04,...,44.5936,44.492,44.3904,44.2888,44.1872,44.0856,43.984,43.8824,43.7808,43.6792
4,Brevard,9,57586.811951,688886.896510,66963.651449,782320.752282,77867.318287,888427.059002,90546.425203,1.008925e+06,...,8.5936,8.492,8.3904,8.2888,8.1872,8.0856,7.984,7.8824,7.7808,7.6792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Union,41,50251.688243,18337.020563,60420.925760,20824.074016,72648.072083,23648.446984,87349.578163,2.685589e+04,...,40.5936,40.492,40.3904,40.2888,40.1872,40.0856,39.984,39.8824,39.7808,39.6792
63,Volusia,8,53911.477972,628620.138334,65460.528314,713880.002672,79483.644826,810703.678004,96510.828089,9.206596e+05,...,7.5936,7.492,7.3904,7.2888,7.1872,7.0856,6.984,6.8824,6.7808,6.6792
64,Wakulla,10,67106.578796,38343.417495,84487.963036,43543.942223,106371.327907,49449.815071,133922.738746,5.615670e+04,...,9.5936,9.492,9.3904,9.2888,9.1872,9.0856,8.984,8.8824,8.7808,8.6792
65,Walton,31,66747.691477,85518.630923,94245.220686,97117.538477,133070.693917,110289.607984,187890.796484,1.252482e+05,...,30.5936,30.492,30.3904,30.2888,30.1872,30.0856,29.984,29.8824,29.7808,29.6792


In [31]:
def sort_columns_by_decade(df, start_year, end_year):
    # Start with fixed columns that are not specific to any decade
    sorted_columns = ['NAME', 'AVG ELEVATION']

    # Add columns for each decade
    for year in range(start_year, end_year + 1, 10):
        # Pattern to identify columns for the specific year
        year_pattern = f"_{year}"
        
        # Get all columns related to this decade and sort them
        decade_columns = [col for col in df.columns if year_pattern in col]
        sorted_decade_columns = sorted(decade_columns, key=lambda x: x.split(year_pattern)[0])

        # Append these sorted columns to the overall list
        sorted_columns.extend(sorted_decade_columns)

    # Reorder the DataFrame using the sorted column list
    return df[sorted_columns]

# Apply the function to final_df
final_df = sort_columns_by_decade(final_df, 2030, 2150)

In [32]:
final_df

,NAME,AVG ELEVATION,Income_2030,Population_2030,elevation_2030,Income_2040,Population_2040,elevation_2040,Income_2050,Population_2050,...,elevation_2120,Income_2130,Population_2130,elevation_2130,Income_2140,Population_2140,elevation_2140,Income_2150,Population_2150,elevation_2150
0,Alachua,33,50475.477283,316236.665772,32.8984,62685.114825,359127.902591,32.7968,77848.171669,407836.485705,...,31.984,4.404741e+05,1.128188e+06,31.8824,5.470214e+05,1.281204e+06,31.7808,6.793418e+05,1.454974e+06,31.6792
1,Baker,40,57178.778569,32091.773339,39.8984,69155.865950,36444.386426,39.7968,83641.762118,41387.345223,...,38.984,3.829802e+05,1.144888e+05,38.8824,4.632021e+05,1.300169e+05,38.7808,5.602278e+05,1.476512e+05,38.6792
2,Bay,13,59267.174711,198980.578127,12.8984,73531.463224,225968.350333,12.7968,91228.848181,256616.479018,...,11.984,5.121581e+05,7.098719e+05,11.8824,6.354232e+05,8.061519e+05,11.7808,7.883554e+05,9.154904e+05,11.6792
3,Bradford,45,52133.398585,32141.741067,44.8984,66086.934008,36501.131286,44.7968,83775.141563,41451.786399,...,43.984,5.586139e+05,1.146671e+05,43.8824,7.081272e+05,1.302194e+05,43.7808,8.976579e+05,1.478810e+05,43.6792
4,Brevard,9,57586.811951,688886.896510,8.8984,66963.651449,782320.752282,8.7968,77867.318287,888427.059002,...,7.984,2.603059e+05,2.457634e+06,7.8824,3.026914e+05,2.790963e+06,7.7808,3.519785e+05,3.169502e+06,7.6792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Union,41,50251.688243,18337.020563,40.8984,60420.925760,20824.074016,40.7968,72648.072083,23648.446984,...,39.984,3.173349e+05,6.541812e+04,39.8824,3.815527e+05,7.429079e+04,39.7808,4.587661e+05,8.436686e+04,39.6792
63,Volusia,8,53911.477972,628620.138334,7.8984,65460.528314,713880.002672,7.7968,79483.644826,810703.678004,...,6.984,3.755467e+05,2.242630e+06,6.8824,4.559972e+05,2.546798e+06,6.7808,5.536821e+05,2.892221e+06,6.6792
64,Wakulla,10,67106.578796,38343.417495,9.8984,84487.963036,43543.942223,9.7968,106371.327907,49449.815071,...,8.984,6.715255e+05,1.367918e+05,8.8824,8.454584e+05,1.553449e+05,8.7808,1.064442e+06,1.764144e+05,8.6792
65,Walton,31,66747.691477,85518.630923,30.8984,94245.220686,97117.538477,30.7968,133070.693917,110289.607984,...,29.984,2.102166e+06,3.050914e+05,29.8824,2.968180e+06,3.464710e+05,29.7808,4.190958e+06,3.934630e+05,29.6792


In [33]:
def calculate_risk_score(decade, df):
    elevation_col = f'elevation_{decade}'
    income_col = f'Income_{decade}'
    population_col = f'Population_{decade}'

    df = df[['NAME', income_col, population_col, elevation_col]].copy()
    
    income_quantiles = df[f'Income_{decade}'].quantile([0.2, 0.4, 0.6, 0.8])
    population_quantiles = df[f'Population_{decade}'].quantile([0.2, 0.4, 0.6, 0.8])
    elevation_quantiles = df[elevation_col].quantile([0.2, 0.4, 0.6, 0.8])

    # Function to categorize values into level scores
    def categorize_into_level_score(value, quantiles):
        if value <= quantiles.iloc[0]:
            return 0
        elif value <= quantiles.iloc[1]:
            return 1
        elif value <= quantiles.iloc[2]:
            return 2
        elif value <= quantiles.iloc[3]:
            return 3
        else:
            return 4
    # Calculate level scores for income, population, and elevation
    df[f'Income_Level_{decade}'] = df[f'Income_{decade}'].apply(categorize_into_level_score, args=(income_quantiles,))
    df[f'Population_Level_{decade}'] = df[f'Population_{decade}'].apply(categorize_into_level_score, args=(population_quantiles,))
    df[f'Elevation_Level_{decade}'] = df[elevation_col].apply(categorize_into_level_score, args=(elevation_quantiles,))
    # Define weights
    weights = {
        'pop': 7.5,  # Weight for population
        'elevation': 10,  # Weight for elevation
        'Income/year': 2.5  # Weight for income
    }
    # Calculate risk score using decade-specific elevation level
    df[f'risk_score_{decade}'] = (
        df[f'Population_Level_{decade}'] * weights['pop'] + 50 -
        df[f'Elevation_Level_{decade}'] * weights['elevation'] -
        df[f'Income_Level_{decade}'] * weights['Income/year']
    )
    return df

In [34]:
for year in range(2030, 2160, 10):
    # Calculate risk score for the given year using final_df
    year_df = calculate_risk_score(str(year), final_df)

    # Merge the calculated scores with final_df
    # The columns in year_df should already align with those in final_df
    final_df = final_df.merge(year_df, on='NAME', how='left')


In [35]:
final_df

,NAME,AVG ELEVATION,Income_2030_x,Population_2030_x,elevation_2030_x,Income_2040_x,Population_2040_x,elevation_2040_x,Income_2050_x,Population_2050_x,...,Population_Level_2140,Elevation_Level_2140,risk_score_2140,Income_2150_y,Population_2150_y,elevation_2150_y,Income_Level_2150,Population_Level_2150,Elevation_Level_2150,risk_score_2150
0,Alachua,33,50475.477283,316236.665772,32.8984,62685.114825,359127.902591,32.7968,77848.171669,407836.485705,...,3,3,35.0,6.793418e+05,1.454974e+06,31.6792,3,3,3,35.0
1,Baker,40,57178.778569,32091.773339,39.8984,69155.865950,36444.386426,39.7968,83641.762118,41387.345223,...,1,4,12.5,5.602278e+05,1.476512e+05,38.6792,2,1,4,12.5
2,Bay,13,59267.174711,198980.578127,12.8984,73531.463224,225968.350333,12.7968,91228.848181,256616.479018,...,2,2,37.5,7.883554e+05,9.154904e+05,11.6792,3,2,2,37.5
3,Bradford,45,52133.398585,32141.741067,44.8984,66086.934008,36501.131286,44.7968,83775.141563,41451.786399,...,1,4,10.0,8.976579e+05,1.478810e+05,43.6792,3,1,4,10.0
4,Brevard,9,57586.811951,688886.896510,8.8984,66963.651449,782320.752282,8.7968,77867.318287,888427.059002,...,4,1,70.0,3.519785e+05,3.169502e+06,7.6792,0,4,1,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Union,41,50251.688243,18337.020563,40.8984,60420.925760,20824.074016,40.7968,72648.072083,23648.446984,...,0,4,7.5,4.587661e+05,8.436686e+04,39.6792,1,0,4,7.5
63,Volusia,8,53911.477972,628620.138334,7.8984,65460.528314,713880.002672,7.7968,79483.644826,810703.678004,...,4,1,65.0,5.536821e+05,2.892221e+06,6.6792,2,4,1,65.0
64,Wakulla,10,67106.578796,38343.417495,9.8984,84487.963036,43543.942223,9.7968,106371.327907,49449.815071,...,1,1,37.5,1.064442e+06,1.764144e+05,8.6792,4,1,1,37.5
65,Walton,31,66747.691477,85518.630923,30.8984,94245.220686,97117.538477,30.7968,133070.693917,110289.607984,...,2,3,25.0,4.190958e+06,3.934630e+05,29.6792,4,2,3,25.0


In [36]:
df_cleaned = final_df.copy()
# Function to remove duplicate columns
def remove_duplicate_columns(df):
    duplicates = []
    for i in range(df.shape[1]):
        col1 = df.iloc[:, i]
        for j in range(i + 1, df.shape[1]):
            col2 = df.iloc[:, j]
            if col1.equals(col2):
                duplicates.append(df.columns.values[j])
    return df.drop(columns=duplicates)

# Remove duplicates
df_cleaned = remove_duplicate_columns(df_cleaned)

In [37]:
df_cleaned

,NAME,AVG ELEVATION,Income_2030_x,Population_2030_x,elevation_2030_x,Income_2040_x,Population_2040_x,elevation_2040_x,Income_2050_x,Population_2050_x,...,Income_Level_2110,risk_score_2110,Income_Level_2120,risk_score_2120,Income_Level_2130,risk_score_2130,Income_Level_2140,risk_score_2140,Income_Level_2150,risk_score_2150
0,Alachua,33,50475.477283,316236.665772,32.8984,62685.114825,359127.902591,32.7968,77848.171669,407836.485705,...,2,37.5,2,37.5,2,37.5,3,35.0,3,35.0
1,Baker,40,57178.778569,32091.773339,39.8984,69155.865950,36444.386426,39.7968,83641.762118,41387.345223,...,2,12.5,2,12.5,2,12.5,2,12.5,2,12.5
2,Bay,13,59267.174711,198980.578127,12.8984,73531.463224,225968.350333,12.7968,91228.848181,256616.479018,...,3,37.5,3,37.5,3,37.5,3,37.5,3,37.5
3,Bradford,45,52133.398585,32141.741067,44.8984,66086.934008,36501.131286,44.7968,83775.141563,41451.786399,...,3,10.0,3,10.0,3,10.0,3,10.0,3,10.0
4,Brevard,9,57586.811951,688886.896510,8.8984,66963.651449,782320.752282,8.7968,77867.318287,888427.059002,...,0,70.0,0,70.0,0,70.0,0,70.0,0,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Union,41,50251.688243,18337.020563,40.8984,60420.925760,20824.074016,40.7968,72648.072083,23648.446984,...,1,7.5,1,7.5,1,7.5,1,7.5,1,7.5
63,Volusia,8,53911.477972,628620.138334,7.8984,65460.528314,713880.002672,7.7968,79483.644826,810703.678004,...,2,65.0,2,65.0,2,65.0,2,65.0,2,65.0
64,Wakulla,10,67106.578796,38343.417495,9.8984,84487.963036,43543.942223,9.7968,106371.327907,49449.815071,...,4,37.5,4,37.5,4,37.5,4,37.5,4,37.5
65,Walton,31,66747.691477,85518.630923,30.8984,94245.220686,97117.538477,30.7968,133070.693917,110289.607984,...,4,25.0,4,25.0,4,25.0,4,25.0,4,25.0


In [38]:
df_cleaned.columns = [col.replace('_x', '') for col in df_cleaned.columns]

In [39]:
for year in range(2030, 2160, 10):
    # Function to classify risk
    def classify_risk(score):
        if score <= 30:
            return 'Low Risk'
        elif score <= 60:
            return 'Medium Risk'
        else:
            return 'High Risk'

    risk_score_column = f'risk_score_{year}'
    risk_class_column = f'Risk_Class_{year}'
    
    # Creating a new column for risk classification
    df_cleaned[risk_class_column] = df_cleaned[risk_score_column].apply(classify_risk)

In [40]:
for year in range(2030, 2160, 10):
    print(df_cleaned[['NAME', f'Risk_Class_{year}']])

          NAME Risk_Class_2030
0      Alachua     Medium Risk
1        Baker        Low Risk
2          Bay     Medium Risk
3     Bradford        Low Risk
4      Brevard       High Risk
..         ...             ...
62       Union        Low Risk
63     Volusia       High Risk
64     Wakulla     Medium Risk
65      Walton        Low Risk
66  Washington        Low Risk

[67 rows x 2 columns]
          NAME Risk_Class_2040
0      Alachua     Medium Risk
1        Baker        Low Risk
2          Bay     Medium Risk
3     Bradford        Low Risk
4      Brevard       High Risk
..         ...             ...
62       Union        Low Risk
63     Volusia       High Risk
64     Wakulla     Medium Risk
65      Walton        Low Risk
66  Washington        Low Risk

[67 rows x 2 columns]
          NAME Risk_Class_2050
0      Alachua     Medium Risk
1        Baker        Low Risk
2          Bay     Medium Risk
3     Bradford        Low Risk
4      Brevard       High Risk
..         ...          

In [41]:
arcGIS_data = pd.read_csv('/Users/ehast/Desktop/risk data/Book1_r.csv')

In [42]:
arcGIS_data

,NAME,Year,Income,Population,elevation,risk_score,risk_level,Recommendation
0,Alachua,2030,5.047548e+04,3.162367e+05,32.8984,60,NaN,NaN
1,Baker,2030,5.717878e+04,3.209177e+04,39.8984,25,NaN,NaN
2,Bay,2030,5.926717e+04,1.989806e+05,12.8984,45,NaN,NaN
3,Bradford,2030,5.213340e+04,3.214174e+04,44.8984,30,NaN,NaN
4,Brevard,2030,5.758681e+04,6.888869e+05,8.8984,70,NaN,NaN
...,...,...,...,...,...,...,...,...
866,Union,2150,4.587661e+05,8.436686e+04,39.6792,25,NaN,NaN
867,Volusia,2150,5.536821e+05,2.892221e+06,6.6792,75,NaN,NaN
868,Wakulla,2150,1.064442e+06,1.764144e+05,8.6792,35,NaN,NaN
869,Walton,2150,4.190958e+06,3.934630e+05,29.6792,35,NaN,NaN


In [43]:
arcGIS_data[arcGIS_data['Year'] == 2030][['NAME','Income']]

,NAME,Income
0,Alachua,50475.47728
1,Baker,57178.77857
2,Bay,59267.17471
3,Bradford,52133.39858
4,Brevard,57586.81195
...,...,...
62,Union,50251.68824
63,Volusia,53911.47797
64,Wakulla,67106.57880
65,Walton,66747.69148


In [44]:
arcGIS_data[arcGIS_data['Year'] == 2030]['elevation'].median()

17.8984

In [45]:
arcGIS_data[arcGIS_data['Year'] == 2030]['elevation'].mean()

21.062579104477617

In [46]:
df_cleaned = sort_columns_by_decade(df_cleaned, 2030, 2150)

In [47]:
df_cleaned[['NAME', 'Risk_Class_2030']][60:67]

,NAME,Risk_Class_2030
60,Suwannee,Low Risk
61,Taylor,Medium Risk
62,Union,Low Risk
63,Volusia,High Risk
64,Wakulla,Medium Risk
65,Walton,Low Risk
66,Washington,Low Risk


In [48]:
arcGIS_data[arcGIS_data['NAME'] == 'Indian River']

,NAME,Year,Income,Population,elevation,risk_score,risk_level,Recommendation
29,Indian River,2030,6.362235e+04,181460.0757,7.8984,45,NaN,NaN
96,Indian River,2040,8.550311e+04,206071.5389,7.7968,45,NaN,NaN
163,Indian River,2050,1.149090e+05,234021.0594,7.6952,45,NaN,NaN
230,Indian River,2060,1.544281e+05,265761.3785,7.5936,45,NaN,NaN
297,Indian River,2070,2.075385e+05,301806.6429,7.4920,45,NaN,NaN
364,Indian River,2080,2.789144e+05,342740.7331,7.3904,45,NaN,NaN
431,Indian River,2090,3.748376e+05,389226.7214,7.2888,45,NaN,NaN
498,Indian River,2100,5.037504e+05,442017.6128,7.1872,45,NaN,NaN
565,Indian River,2110,6.769984e+05,501968.5425,7.0856,45,NaN,NaN
632,Indian River,2120,9.098293e+05,570050.6277,6.9840,45,NaN,NaN


In [49]:
arcGIS_data.at[29, 'NAME'] = 'IndianRiver'
arcGIS_data.at[96, 'NAME'] = 'IndianRiver'
arcGIS_data.at[163, 'NAME'] = 'IndianRiver'
arcGIS_data.at[230, 'NAME'] = 'IndianRiver'
arcGIS_data.at[297, 'NAME'] = 'IndianRiver'
arcGIS_data.at[364, 'NAME'] = 'IndianRiver'
arcGIS_data.at[431, 'NAME'] = 'IndianRiver'
arcGIS_data.at[498, 'NAME'] = 'IndianRiver'
arcGIS_data.at[565, 'NAME'] = 'IndianRiver'
arcGIS_data.at[632, 'NAME'] = 'IndianRiver'
arcGIS_data.at[699, 'NAME'] = 'IndianRiver'
arcGIS_data.at[766, 'NAME'] = 'IndianRiver'
arcGIS_data.at[833, 'NAME'] = 'IndianRiver'

In [50]:
# Step 1: Extract Year and Create a New DataFrame for Each Year
years = range(2030, 2160, 10)  # Adjust range as needed
df_cleaned_list = []

for year in years:
    temp_df = df_cleaned[['NAME', f'Risk_Class_{year}']].copy()
    temp_df['Year'] = year
    temp_df.rename(columns={f'Risk_Class_{year}': 'Risk_Class'}, inplace=True)
    df_cleaned_list.append(temp_df)

# Combine all yearly DataFrames
df_cleaned_combined = pd.concat(df_cleaned_list)

# Step 2: Merge DataFrames
fin_df = pd.merge(arcGIS_data, df_cleaned_combined, left_on=['NAME', 'Year'], right_on=['NAME', 'Year'])

# Step 3: Update risk_level
fin_df['risk_level'] = fin_df['Risk_Class']

# Step 4: Drop Unnecessary Columns
final_df = fin_df.drop(columns=['Risk_Class'])

# Now final_df contains the updated information


In [51]:
final_df[20:30]

,NAME,Year,Income,Population,elevation,risk_score,risk_level,Recommendation
20,Glades,2030,47315.08984,1.377065e+04,11.8984,40,Medium Risk,NaN
21,Gulf,2030,49809.70723,1.611686e+04,4.8984,45,Medium Risk,NaN
22,Hamilton,2030,45715.36241,1.590337e+04,36.8984,25,Low Risk,NaN
23,Hardee,2030,43267.10549,2.876211e+04,7.8984,55,Medium Risk,NaN
24,Hendry,2030,40351.52909,4.499253e+04,7.8984,55,Medium Risk,NaN
25,Hernando,2030,51110.84906,2.208971e+05,17.8984,60,Medium Risk,NaN
26,Highlands,2030,40676.28995,1.149655e+05,21.8984,60,Medium Risk,NaN
27,Hillsborough,2030,64031.62779,1.657750e+06,19.8984,60,Medium Risk,NaN
28,Holmes,2030,40401.40898,2.231854e+04,38.8984,30,Low Risk,NaN
29,IndianRiver,2030,63622.34531,1.814601e+05,7.8984,45,Medium Risk,NaN


In [52]:
scoring_data = pd.read_csv('/Users/ehast/Desktop/risk data/scoring_data.csv')

In [53]:
scoring_data.at[15, 'NAME'] = 'IndianRiver'
scoring_data[scoring_data['NAME']=='IndianRiver']

,NAME,LEVEL1_L_1,count,Score
15,IndianRiver,Urban and Built-Up,1663,16


In [54]:
final_df = pd.merge(final_df, scoring_data[['NAME', 'Score']], on='NAME', how='left')

In [55]:
final_df

,NAME,Year,Income,Population,elevation,risk_score,risk_level,Recommendation,Score
0,Alachua,2030,5.047548e+04,3.162367e+05,32.8984,60,Medium Risk,NaN,65
1,Baker,2030,5.717878e+04,3.209177e+04,39.8984,25,Low Risk,NaN,9
2,Bay,2030,5.926717e+04,1.989806e+05,12.8984,45,Medium Risk,NaN,55
3,Bradford,2030,5.213340e+04,3.214174e+04,44.8984,30,Low Risk,NaN,32
4,Brevard,2030,5.758681e+04,6.888869e+05,8.8984,70,High Risk,NaN,42
...,...,...,...,...,...,...,...,...,...
866,Union,2150,4.587661e+05,8.436686e+04,39.6792,25,Low Risk,NaN,17
867,Volusia,2150,5.536821e+05,2.892221e+06,6.6792,75,High Risk,NaN,47
868,Wakulla,2150,1.064442e+06,1.764144e+05,8.6792,35,Medium Risk,NaN,36
869,Walton,2150,4.190958e+06,3.934630e+05,29.6792,35,Low Risk,NaN,63


In [56]:
# High score is most number of urban area
# Low Risk <50 quantile: Low risk, Low urban area

# Recommendation Rules

# Low Risk, <50 quantile: No action Recommended at this time
# Low Risk, >50 quantile: Dialogue with county officials, risk is low at this time.
# Medium Risk, <50 quantile: Be careful around rivers and swamp areas due to potential flooding.
# Medium Risk >50 quantile: Avoid low elevation areas to live
# High Risk <50 quantile: Move to higher ground, avoid low areas and swamp areas.
# High Risk >50 quantile: Avoid at all costs. Move if necessary. Sea level rise will significantly impact all areas of life.

In [57]:
# Step 1: Calculate the 50th Quantile of the Score Column
median_score = final_df['Score'].median()

# Step 2: Map Recommendations
def map_recommendations(row):
    if row['risk_level'] == 'Low Risk':
        if row['Score'] < median_score:
            return 'No action recommended at this time.'
        else:
            return 'Dialogue with county officials, risk is low at this time.'

    elif row['risk_level'] == 'Medium Risk':
        if row['Score'] < median_score:
            return 'Be careful around rivers and swamp areas due to potential flooding.'
        else:
            return 'Avoid low elevation areas to live.'

    elif row['risk_level'] == 'High Risk':
        if row['Score'] < median_score:
            return 'Move to higher ground, avoid low areas and swamp areas.'
        else:
            return 'Avoid at all costs. Move if necessary. Sea level rise will significantly impact all areas of life.'

    return None  # Default case if none of the above conditions are met

# Apply the function to each row
final_df['Recommendation'] = final_df.apply(map_recommendations, axis=1)

# Now the Recommendation column in final_df should be updated based on the rules


In [58]:
final_df

,NAME,Year,Income,Population,elevation,risk_score,risk_level,Recommendation,Score
0,Alachua,2030,5.047548e+04,3.162367e+05,32.8984,60,Medium Risk,Avoid low elevation areas to live.,65
1,Baker,2030,5.717878e+04,3.209177e+04,39.8984,25,Low Risk,No action recommended at this time.,9
2,Bay,2030,5.926717e+04,1.989806e+05,12.8984,45,Medium Risk,Avoid low elevation areas to live.,55
3,Bradford,2030,5.213340e+04,3.214174e+04,44.8984,30,Low Risk,No action recommended at this time.,32
4,Brevard,2030,5.758681e+04,6.888869e+05,8.8984,70,High Risk,Avoid at all costs. Move if necessary. Sea lev...,42
...,...,...,...,...,...,...,...,...,...
866,Union,2150,4.587661e+05,8.436686e+04,39.6792,25,Low Risk,No action recommended at this time.,17
867,Volusia,2150,5.536821e+05,2.892221e+06,6.6792,75,High Risk,Avoid at all costs. Move if necessary. Sea lev...,47
868,Wakulla,2150,1.064442e+06,1.764144e+05,8.6792,35,Medium Risk,Avoid low elevation areas to live.,36
869,Walton,2150,4.190958e+06,3.934630e+05,29.6792,35,Low Risk,"Dialogue with county officials, risk is low at...",63


In [59]:
# Group by Year and calculate median elevation for each group
median_elevations_per_decade = final_df.groupby('Year')['elevation'].median()

# median_elevations_per_decade now contains the median elevation for each decade


In [60]:
# Convert the Series to a DataFrame
median_elevations_df = median_elevations_per_decade.reset_index()

# Renaming columns for clarity
median_elevations_df.columns = ['Year', 'Median Elevation']

# Now median_elevations_df is a DataFrame with each decade and its median elevation


In [61]:
median_elevations_df

,Year,Median Elevation
0,2030,17.8984
1,2040,17.7968
2,2050,17.6952
3,2060,17.5936
4,2070,17.4920
5,2080,17.3904
6,2090,17.2888
7,2100,17.1872
8,2110,17.0856
9,2120,16.9840


In [62]:
# Calculate median elevations per decade
median_elevations_per_decade = final_df.groupby('Year')['elevation'].median().reset_index()
median_elevations_per_decade.rename(columns={'elevation': 'Median Elevation'}, inplace=True)

# Merge with final_df
final_df = pd.merge(final_df, median_elevations_per_decade, on='Year', how='left')

# final_df_with_median_elevation now contains the original data along with the median elevation for each decade


In [63]:
final_df[final_df['NAME']=='IndianRiver']

,NAME,Year,Income,Population,elevation,risk_score,risk_level,Recommendation,Score,Median Elevation
29,IndianRiver,2030,6.362235e+04,181460.0757,7.8984,45,Medium Risk,Be careful around rivers and swamp areas due t...,16,17.8984
96,IndianRiver,2040,8.550311e+04,206071.5389,7.7968,45,Medium Risk,Be careful around rivers and swamp areas due t...,16,17.7968
163,IndianRiver,2050,1.149090e+05,234021.0594,7.6952,45,Medium Risk,Be careful around rivers and swamp areas due t...,16,17.6952
230,IndianRiver,2060,1.544281e+05,265761.3785,7.5936,45,Medium Risk,Be careful around rivers and swamp areas due t...,16,17.5936
297,IndianRiver,2070,2.075385e+05,301806.6429,7.4920,45,Medium Risk,Be careful around rivers and swamp areas due t...,16,17.4920
364,IndianRiver,2080,2.789144e+05,342740.7331,7.3904,45,Medium Risk,Be careful around rivers and swamp areas due t...,16,17.3904
431,IndianRiver,2090,3.748376e+05,389226.7214,7.2888,45,Medium Risk,Be careful around rivers and swamp areas due t...,16,17.2888
498,IndianRiver,2100,5.037504e+05,442017.6128,7.1872,45,Medium Risk,Be careful around rivers and swamp areas due t...,16,17.1872
565,IndianRiver,2110,6.769984e+05,501968.5425,7.0856,45,Medium Risk,Be careful around rivers and swamp areas due t...,16,17.0856
632,IndianRiver,2120,9.098293e+05,570050.6277,6.9840,45,Medium Risk,Be careful around rivers and swamp areas due t...,16,16.9840


In [64]:
#df_cleaned.to_csv('/Users/ehast/Desktop/risk data/full_data.csv', index=False)

In [66]:
#final_df.to_csv('/Users/ehast/Desktop/risk data/final_recommendation_data.csv', index=False)